# Descirpion
The purpose of this notebook is to take the "cleaned" csv file as an input, and then further refine/split the data for future SQL tables creation. 

---

# About the data
This dataset (Ask A Manager Salary Survey 2021 dataset) contains salary information by industry, age group, location, gender, years of experience, and education level. The data is based on approximately 28k user entered responses.

**Features:**
- `timestamp` - time when the survey was filed
- `age` - Age range of the person
- `industry` - Working industry
- `job_title` - Job title
- `job_context` - Additional context for the job title
- `annual_salary` - Annual salary
- `additional_salary` - Additional monetary compensation
- `currency` - Salary currency
- `currency_context` - Other currency
- `salary_context` - Additional context for salary
- `country` -  Country in which person is working
- `state` - State in which person is working
- `city` - City in which person is working
- `total_experience` -  Year  range of total work experience
- `current_experience` - Year range of current field  work experience
- `education` - Highest level of education completed
- `gender` - Gender of the person
- `race` - Race of the person

# Reading the file

In [1]:
import pandas as pd

data = pd.read_csv('Data/salary_responses_clean.csv')

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27848 entries, 0 to 27847
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   timestamp           27848 non-null  object 
 1   age                 27848 non-null  object 
 2   industry            27778 non-null  object 
 3   job_title           27848 non-null  object 
 4   job_context         7204 non-null   object 
 5   annual_salary       27848 non-null  int64  
 6   additional_salary   20634 non-null  float64
 7   currency            27848 non-null  object 
 8   currency_context    191 non-null    object 
 9   salary_context      3026 non-null   object 
 10  country             27848 non-null  object 
 11  state               22894 non-null  object 
 12  city                27773 non-null  object 
 13  total_experience    27848 non-null  object 
 14  current_experience  27848 non-null  object 
 15  education           27638 non-null  object 
 16  gend

# Categorical data
Let's handle the categorical data first.

**Categorical features include:**
- `age`
- `total_experience`
- `current_experience`
- `education`
- `gender`


*The `state`, `race` and `currency` attributes can also be considered categorical, but more cleaning need to be done. We will leave it for now.*

## Age

In [3]:
data['age'].value_counts()

25-34         12562
35-44          9853
45-54          3171
18-24          1173
55-64           986
65 or over       92
under 18         11
Name: age, dtype: int64

In [4]:
data['age'].isnull().sum()

0

Lets create new columns `age_min` and `age_max` so we could more easily analyze the data.

In [5]:
import numpy as np

In [6]:
def age_range_to_min(row):
    age_range = row['age']
    
    if '-' in age_range:
        age_min = age_range.split('-')[0]
    elif 'over' in age_range:
        age_min = age_range.split()[0]
    elif 'under' in age_range:
        return np.nan
    
    return int(age_min)

def age_range_to_max(row):
    age_range = row['age']
    
    if '-' in age_range:
        age_max = age_range.split('-')[1]
    elif 'over' in age_range:
        return np.nan
    elif 'under' in age_range:
        age_max = age_range.split()[-1]
    
    return int(age_max)

In [7]:
data['age_min'] = data.apply(lambda row: age_range_to_min(row), axis=1)
data['age_max'] = data.apply(lambda row: age_range_to_max(row), axis=1)

## Experience
Same goes for `total_experience` and `current_experience` attributes.

In [8]:
data['total_experience'].value_counts()

11 - 20 years       9579
8 - 10 years        5348
5-7 years           4843
21 - 30 years       3617
2 - 4 years         2974
31 - 40 years        863
1 year or less       504
41 years or more     120
Name: total_experience, dtype: int64

In [9]:
data['total_experience'].isnull().sum()

0

In [10]:
def experience_range_to_min(row, attribute):
    total_exp_range = row[attribute]
    
    if '-' in total_exp_range:
        total_exp_min = total_exp_range.strip().split('-')[0]
    elif 'more' in total_exp_range:
        total_exp_min = total_exp_range.split()[0]
    elif 'less' in total_exp_range:
        return np.nan
    
    return int(total_exp_min)

def experience_range_to_max(row, attribute):
    total_exp_range = row[attribute]
    
    if '-' in total_exp_range:
        total_exp_max = total_exp_range.strip().replace('years', '').split('-')[1]
    elif 'more' in total_exp_range:
        return np.nan
    elif 'less' in total_exp_range:
        total_exp_max = total_exp_range.split()[0]
    
    return int(total_exp_max)

In [11]:
data['total_experience_min'] = data.apply(lambda row: experience_range_to_min(row, 'total_experience'), axis=1)
data['total_experience_max'] = data.apply(lambda row: experience_range_to_max(row, 'total_experience'), axis=1)

In [12]:
data['current_experience'].value_counts()

11 - 20 years       6514
5-7 years           6485
2 - 4 years         6187
8 - 10 years        4945
21 - 30 years       1863
1 year or less      1438
31 - 40 years        378
41 years or more      38
Name: current_experience, dtype: int64

In [13]:
data['current_experience_min'] = data.apply(lambda row: experience_range_to_min(row, 'current_experience'), axis=1)
data['current_experience_max'] = data.apply(lambda row: experience_range_to_max(row, 'current_experience'), axis=1)

## Education

In [14]:
data['education'].value_counts()

College degree                        13414
Master's degree                        8814
Some college                           2039
PhD                                    1420
Professional degree (MD, JD, etc.)     1319
High School                             632
Name: education, dtype: int64

Clean up naming:

In [15]:
data['education'].replace({"Professional degree (MD, JD, etc.)": "Professional degree"}, inplace=True)

It would be nice to have some kind of knowledge about the actual "level" of education (e.g. 0 - High school, 1 - Some college, etc.). Lets map those values to their level:

In [16]:
data['education_lvl'] = data['education'].map({'High School': 1, 'Some college': 2, 'College degree': 3, "Master's degree": 4, 'Professional degree': 5})

In [17]:
data[['education', 'education_lvl']].head()

,education,education_lvl
0,Master's degree,4.0
1,College degree,3.0
2,College degree,3.0
3,College degree,3.0
4,College degree,3.0


## Gender

In [18]:
data['gender'].value_counts()

Woman         21256
Man            5398
Non-binary      739
Other           290
Name: gender, dtype: int64

Clean up naming:

In [19]:
data['gender'].replace({"Other or prefer not to answer": "Other"}, inplace=True)

Lets create some kind of mapping so it could be easier to use in SQL queries:

In [20]:
data['gender_idx'] = data['gender'].map({'Woman': 1, 'Man': 2, 'Non-binary': 3, "Other": 4})

In [21]:
data[['gender', 'gender_idx']].head()

,gender,gender_idx
0,Woman,1.0
1,Non-binary,3.0
2,Woman,1.0
3,Woman,1.0
4,Woman,1.0


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27848 entries, 0 to 27847
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   timestamp               27848 non-null  object 
 1   age                     27848 non-null  object 
 2   industry                27778 non-null  object 
 3   job_title               27848 non-null  object 
 4   job_context             7204 non-null   object 
 5   annual_salary           27848 non-null  int64  
 6   additional_salary       20634 non-null  float64
 7   currency                27848 non-null  object 
 8   currency_context        191 non-null    object 
 9   salary_context          3026 non-null   object 
 10  country                 27848 non-null  object 
 11  state                   22894 non-null  object 
 12  city                    27773 non-null  object 
 13  total_experience        27848 non-null  object 
 14  current_experience      27848 non-null

# Address attributes

In [23]:
empty_value = np.nan

## City

In [24]:
data['city'].value_counts()

Boston          770
Chicago         748
New York        710
Seattle         691
London          569
               ... 
Southwest MI      1
Ystad             1
Utica             1
Howell            1
Pontiac           1
Name: city, Length: 4631, dtype: int64

In [30]:
try:
    from geopy import Nominatim
except:
    !pip install geopy
    from geopy import Nominatim

geolocator = Nominatim(user_agent="cs513-final-project")

In [31]:
try:
    from geotext import GeoText
except:
    !pip install geotext
    from geotext import GeoText
    

def get_city_from_text(row):
    city = row['city']

    if type(city) != str:
        return empty_value
        
    if city.strip().upper() == "REMOTE":
        return "Remote"
    
    places = GeoText(city)
    
    if(len(places.cities) > 0):
        return places.cities[0]
    
    location = geolocator.geocode(city, exactly_one=True, addressdetails=True, timeout=10)

    if location != None:
        location_keys = location.raw['address'].keys()
        if "town" in location_keys:
            return location.raw['address']['town']
        elif "city" in location_keys:
            return location.raw['address']['city']
        elif "hamlet" in location_keys:
            return location.raw['address']['hamlet']
        elif "village" in location_keys:
            return location.raw['address']['village']
        elif "place" in location_keys:
            return location.raw['address']['place']
        elif "municipality" in location_keys:
            return location.raw['address']['municipality']
        elif "township" in location_keys:
            return location.raw['address']['township']
        elif "county" in location_keys:
            return location.raw['address']['county']
        
    return empty_value 

In [32]:
data['city'] = data.apply(lambda row: get_city_from_text(row), axis=1)
data['city'].value_counts()

New York          1327
Washington        1009
Boston             872
Chicago            870
Seattle            760
                  ... 
Tijuana              1
Calamba              1
West Greenwich       1
Uintah County        1
Midwest              1
Name: city, Length: 2591, dtype: int64

## Country

In [33]:
try:
    import pycountry
except:
    !pip install pycountry
    import pycountry

    
def get_country_from_text(row):
    country = row['country']

    if type(country) != str:
        return empty_value
    
    if country.strip().upper() == "AMERICA":
        return "United States"
  
    places = GeoText(country)
    
    if(len(places.countries) > 0):
        return places.countries[0]
    
    location = geolocator.geocode(country, exactly_one=True, addressdetails=True, timeout=10)

    if location != None:
        if "country" in location.raw['address'].keys():
            country_code = location.raw['address']['country_code']
            return pycountry.countries.get(alpha_2=country_code).name
    
    return empty_value 

In [ ]:
data['country'] = data.apply(lambda row: get_country_from_text(row), axis=1)

## State

Next, let's examine rows with multiple `state` values and a `city` value containing only one word (not Remote). For each row, we will attempt to match the city to one of the states in the column. To do so, we will use the **geolocator** module

In [48]:
try:
    from geopy.geocoders import Nominatim   
except ImportError:
    !pip install geopy
    from geopy.geocoders import Nominatim  
    
    
def get_state_from_text(row):
    states = row['state']
    country = row['country']
    city = row['city']
    
    if country == empty_value or country != "United States":
        return empty_value
    
    if type(states) != str:
        return empty_value
    
    if city == empty_value or city == "Remote":
        return empty_value
    
    if ',' not in states:
        return states
    
    states = [x.strip() for x in states.split(',')]
    
    for state in states:
        lookup = f"{city}, {state}, {country}"
        
        location = geolocator.geocode(lookup, exactly_one = True, addressdetails = True, timeout=10)

        if location != None:
            if location.raw['address']['country'] == country:
                return location.raw['address']['state']
        
    return empty_value 

In [49]:
data['state'] = data.apply(lambda row: get_state_from_text(row), axis=1)
data['state'].value_counts()

Unknown                 5648
California              2562
New York                2116
Massachusetts           1473
Texas                   1227
Illinois                1177
Washington              1156
District of Columbia     974
Pennsylvania             909
Virginia                 766
Minnesota                697
Ohio                     635
Oregon                   614
Colorado                 609
North Carolina           579
Maryland                 537
Georgia                  527
Michigan                 522
Florida                  504
Wisconsin                450
New Jersey               367
Missouri                 328
Indiana                  319
Arizona                  303
Tennessee                269
Connecticut              215
Utah                     198
Kentucky                 188
Iowa                     172
Kansas                   149
South Carolina           136
Louisiana                125
Alabama                  115
Maine                    115
Oklahoma      

## Continent
Based on country value we can add new attribue `continent`

In [50]:
try:
    from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
except:
    !pip install pycountry-convert
    from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

    
continent_map = {
    'AF': 'Africa',
    'NA': 'North America',
    'OC': 'Oceania',
    'AN': 'Antarctica',
    'AS': 'Asia',
    'EU': 'Europe',
    'SA': 'South America',
}
    
    
def get_continent_from_country(row):
    country = row['country']
    
    if country == empty_value:
        return empty_value
    
    country_code =  country_name_to_country_alpha2(country)
    continent_code = country_alpha2_to_continent_code(country_code)
    
    return continent_map[continent_code]

In [51]:
data['continent'] = data.apply(lambda row: get_continent_from_country(row), axis=1)
data['continent'].value_counts()

North America    24732
Europe            2363
Oceania            518
Asia               150
Africa              34
Unknown             26
South America       25
Name: continent, dtype: int64

## Latitude, Longitude

In [77]:
def get_lat_long_from_full_address(city, country):
    if city == empty_value or country == empty_value:
        return empty_value
    
    full_address = f"{city}, {country}"
    
    location = geolocator.geocode(full_address, exactly_one=True, addressdetails=True, timeout=10)
    if location != None:
        return (location.latitude, location.longitude)
    else:
        return empty_value

In [79]:
data['lat_long'] = data.apply(lambda row: get_lat_long_from_full_address(row['city'], row['country']), axis=1)

In [100]:
data[['lat', 'long']] = pd.DataFrame(data['lat_long'].tolist(), index=data.index)

# Text data
Let's handle the text data.

**Text features include:**
- `industry`
- `job_title`
- `job_context`
- `salary_context`

## Context
Both `salary_context` and `job_context` features have too much information. It is basically a plain text provided by the user. As it does not help our analysis, we decided to drop those columns.

In [101]:
data.drop(labels=['salary_context', 'job_context'], axis=1, inplace=True)

## Industry and job title

In [103]:
data['job_title'].value_counts()

Software Engineer                                   390
Project Manager                                     283
Senior Software Engineer                            232
Director                                            219
Executive Assistant                                 186
                                                   ... 
Senior User Experience Researcher                     1
Process Improvement Consultant                        1
Assistant Director - Medical Library                  1
Expert                                                1
Associate Director of Recruitment and Engagement      1
Name: job_title, Length: 14208, dtype: int64

# Currency
Here we have a little bit more work to do. The currency of the salary is defined by the `currency` attribute, but sometimes it can be also defined by the `currency_context`. We need to clean those 2 columns and merge them into one. 

## Merge columns 

In [104]:
data[['currency', 'currency_context']]

,currency,currency_context
0,USD,NaN
1,GBP,NaN
2,USD,NaN
3,USD,NaN
4,USD,NaN
...,...,...
27843,USD,NaN
27844,AUD/NZD,NaN
27845,USD,NaN
27846,Other,NGN


In [105]:
data['currency'].value_counts()

USD        23214
CAD         1659
GBP         1578
EUR          632
AUD/NZD      500
Other        150
CHF           37
SEK           37
JPY           23
ZAR           14
HKD            4
Name: currency, dtype: int64

In [106]:
data['currency_context'].value_counts()

USD                              11
NOK                              10
INR                               9
MYR                               8
DKK                               8
                                 ..
DKK                               1
Php                               1
CAD                               1
Philippine Peso                   1
6000 in stock grants annually     1
Name: currency_context, Length: 114, dtype: int64

As we can see, when the `currency` feature has value of 'Other', then the currency is defined by `currency_context`. Lets clean this up

In [107]:
data['currency'] = np.where(data["currency"] == "Other", data['currency_context'], data["currency"])

In [108]:
data.drop(labels=['currency_context'], axis=1, inplace=True)

## Handle AUD/NZD values

In [120]:
def split_currencies(row):
    currency = row['currency']
    
    if currency != 'AUD/NZD':
        return currency

    country = row['country']
    
    if country == 'Australia':
        return 'AUD'
    if country == 'New Zealand':
        return 'NZD'
    return np.nan

In [121]:
data['currency'] = data.apply(lambda row: split_currencies(row), axis=1)

## Clean manually 

In [122]:
data['currency'][data['currency'].str.len() > 3].value_counts()

ILS/NIS                     1
Indian rupees               1
Philippine Peso             1
Rupees                      1
Argentine Peso              1
US Dollar                   1
DKK                         1
SGD                         1
Mexican pesos               1
Korean Won                  1
Equity                      1
American Dollars            1
THAI  BAHT                  1
Taiwanese dollars           1
PLN (Polish zloty)          1
Peso Argentino              1
Singapore Dollara           1
RMB (chinese yuan)          1
Norwegian kroner (NOK)      1
Euro                        1
Philippine peso (PHP)       1
Danish Kroner               1
Israeli Shekels             1
Thai Baht                   1
KRW (Korean Won)            1
Philippine Pesos            1
ILS (Shekel)                1
NIS (new Israeli shekel)    1
IDR                         1
BRL (R$)                    1
PhP (Philippine Peso)       1
INR (Indian Rupee)          1
Australian Dollars          1
Argentinia

In [133]:
messy_currencies = data['currency'][data['currency'].str.len() > 3].to_list()
right_currencies = ['ARP', 'INR', 'BRL', 'MXN', 'USD', 'PLN', 'CZK', 'NOK', 'ILS', 'USD', 'NIS', 'RMB', 'TWD', 'PHP', 'KRW', 'IDR', 'ILS', 'DKK', 'RMB', 'AUD', 'PLN', 'PHP', 'AUD', np.nan, 'ARS', 'ILS', 'PHP', 'ARP', 'PHP', 'INR', 'DKK', 'KRW', 'EUR', 'SGD', 'MXN', 'THB', 'THB', 'HRK', 'PLN', 'INR', 'SGD']

data['currency'] = data['currency'].replace(messy_currencies, right_currencies)

## USD rate
To have a consistent analysis for the salary values, we need to have only one currency (e.g. USD).

In [142]:
!pip install forex_python
from forex_python.converter import CurrencyRates

You should consider upgrading via the 'c:\users\wiewi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [143]:
currency_rates = CurrencyRates()

In [144]:
from datetime import timedelta


def to_USD_rate(row):
    currency = row['currency']
    datatime = pd.to_datetime(row['timestamp'])
    
    try:
        return currency_rates.get_rate(currency, 'USD', datatime)
    except:
        return np.nan

In [ ]:
data['USD_rate'] = data.apply(lambda row: to_USD_rate(row), axis=1)

In [ ]:
data[['currency', 'USD_rate']]

In [ ]:
data['currency'][data['USD_rate'] == np.nan]

# Numeric data
Lets handle numeric data now.

**Numeric features include:**
- `annual_salary`
- `additional_salary`

## Annual salary

In [ ]:
data['annual_salary']